## Chapter 3: Dice functions

Icepool has a variety of built-in functions and methods. You can consult the [API documentation](https://highdiceroller.github.io/icepool/apidoc/icepool.html#Die) for details. We won't cover every single one in this chapter, but we will cover some of the more important ones.

### Free functions

#### `highest` and `lowest`

The `highest` function takes the highest of several dice rolls.

In [1]:
import piplite
await piplite.install("icepool")

from icepool import d6, highest, lowest, highest, lowest

print(highest(d6, d6))

Die with denominator 36

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        1 |   2.777778% |
|       2 |        3 |   8.333333% |
|       3 |        5 |  13.888889% |
|       4 |        7 |  19.444444% |
|       5 |        9 |  25.000000% |
|       6 |       11 |  30.555556% |




You can use `highest` and the `keep` and `drop` arguments to keep multiple dice and sum them.

In [2]:
print(highest(d6, d6, d6, d6, keep=3))  # 4d6 drop lowest

Die with denominator 1296

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       3 |        1 |   0.077160% |
|       4 |        4 |   0.308642% |
|       5 |       10 |   0.771605% |
|       6 |       21 |   1.620370% |
|       7 |       38 |   2.932099% |
|       8 |       62 |   4.783951% |
|       9 |       91 |   7.021605% |
|      10 |      122 |   9.413580% |
|      11 |      148 |  11.419753% |
|      12 |      167 |  12.885802% |
|      13 |      172 |  13.271605% |
|      14 |      160 |  12.345679% |
|      15 |      131 |  10.108025% |
|      16 |       94 |   7.253086% |
|      17 |       54 |   4.166667% |
|      18 |       21 |   1.620370% |




In [3]:
print(highest(d6, d6, d6, keep=1, drop=1))  # middle of 3d6: drop highest, then keep the next

Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |       16 |   7.407407% |
|       2 |       40 |  18.518519% |
|       3 |       52 |  24.074074% |
|       4 |       52 |  24.074074% |
|       5 |       40 |  18.518519% |
|       6 |       16 |   7.407407% |




#### `apply`

`apply` works similarly to how AnyDice expands dice arguments to functions: it evaluates a function over to all possible joint outcomes of one or more dice, then constructs a new die from the results.

Here we use *Ironsworn* as an example:

* Roll an action die (d6 + modifier) and two challenge dice (d10s).
* The result is how many challenge dice are beaten by the action die.

In [4]:
from icepool import apply, d6, d10

def ironsworn(action, modifier, challenge_1, challenge_2):
    return (action + modifier > challenge_1) + (action + modifier > challenge_2)

print(apply(ironsworn, d6, 1, d10, d10))

Die with denominator 600

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |      271 |  45.166667% |
|       1 |      238 |  39.666667% |
|       2 |       91 |  15.166667% |




Note that if we had done `(d6 + 1 > d10) + (d6 + 1 > d10)` we would get the wrong result because it would treat this as rolling the d6 independently against each of the two challenge dice. Thus, `apply` can be used to "freeze" the roll of individual dice so that you can use the same roll twice or in control flow (e.g. if-else statements).

`apply` is not especially efficient. If possible, using a `Pool` will tend to be considerably faster. More on this in the next chapter.

### `Die` methods

#### `simplify`

A `Die` is a non-normalized probability distribution that assigns an integer quantity to each outcome. You might think of a `Die` as a set of fractions where the denominator is the total quantity in the die.

Most operations do not put a `Die` into simplest form automatically; this allows you to track the "raw" denominator of a series of operations. If you want to put a `Die` into simplest form, you can use the `simplify` method:

In [5]:
print(d6 == d6)
print((d6 == d6).simplify())

DieWithTruth with denominator 36

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| False   |       30 |  83.333333% |
| True    |        6 |  16.666667% |


Die with denominator 6

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| False   |        5 |  83.333333% |
| True    |        1 |  16.666667% |




#### `reroll`

This allows you to reroll selected outcomes of a `Die`. For example, this rerolls 1s and 2s on a d6:

In [6]:
print(d6.reroll([1, 2]))

Die with denominator 4

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       3 |        1 |  25.000000% |
|       4 |        1 |  25.000000% |
|       5 |        1 |  25.000000% |
|       6 |        1 |  25.000000% |




By default, there is no limit to the number of times rerolled. If you want to set a limit, use the `depth` argument:

In [7]:
print(d6.reroll([1, 2], depth=1))

Die with denominator 36

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |        2 |   5.555556% |
|       2 |        2 |   5.555556% |
|       3 |        8 |  22.222222% |
|       4 |        8 |  22.222222% |
|       5 |        8 |  22.222222% |
|       6 |        8 |  22.222222% |




#### `explode`

This explodes the given outcomes, that is, if the outcome is rolled, the die is rolled again and the result added to the running total. By default the single highest outcome is exploded. Like `Die.reroll()`, `explode()` has a depth, though its default is finite.

In [8]:
print(d6.explode(depth=2))

Die with denominator 216

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       1 |       36 |  16.666667% |
|       2 |       36 |  16.666667% |
|       3 |       36 |  16.666667% |
|       4 |       36 |  16.666667% |
|       5 |       36 |  16.666667% |
|       7 |        6 |   2.777778% |
|       8 |        6 |   2.777778% |
|       9 |        6 |   2.777778% |
|      10 |        6 |   2.777778% |
|      11 |        6 |   2.777778% |
|      13 |        1 |   0.462963% |
|      14 |        1 |   0.462963% |
|      15 |        1 |   0.462963% |
|      16 |        1 |   0.462963% |
|      17 |        1 |   0.462963% |
|      18 |        1 |   0.462963% |




#### `map`

Similar to calling `apply` on the `Die`, but with some additional features:

* Instead of calling a function on each outcome of the die, you can supply a `dict` mapping old outcomes to new outcomes.
* You can repeat the mapping by providing the `repeat` argument.
  * Experimental: You can even effectively repeat an infinite number of times by using `repeat=None`. This can be used to model processes that almost surely terminate, as long as they have a reasonably small state space.

Here are some examples:

In [9]:
# Replace numbers with letters.
print(d6.map({1:'a', 2:'b', 3:'c', 4:'d', 5:'e', 6:'f'}))

Die with denominator 6

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| a       |        1 |  16.666667% |
| b       |        1 |  16.666667% |
| c       |        1 |  16.666667% |
| d       |        1 |  16.666667% |
| e       |        1 |  16.666667% |
| f       |        1 |  16.666667% |




In [10]:
# Roll d6s until the sum is at least 10; the result is the total.
# Note that you are allowed to return a die from the function sent to `Die.map()`
# (or `apply()` for that matter),
print(d6.map(lambda x: x if x >= 10 else x + d6))

Die with denominator 36

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       2 |        1 |   2.777778% |
|       3 |        2 |   5.555556% |
|       4 |        3 |   8.333333% |
|       5 |        4 |  11.111111% |
|       6 |        5 |  13.888889% |
|       7 |        6 |  16.666667% |
|       8 |        5 |  13.888889% |
|       9 |        4 |  11.111111% |
|      10 |        3 |   8.333333% |
|      11 |        2 |   5.555556% |
|      12 |        1 |   2.777778% |




#### `highest` and `lowest`

Similar to the free functions of the same name, this is convenient if all the dice you roll are the same. Here's another way to do 4d6 drop lowest:

In [11]:
print(d6.highest(4, 3))

Die with denominator 1296

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       3 |        1 |   0.077160% |
|       4 |        4 |   0.308642% |
|       5 |       10 |   0.771605% |
|       6 |       21 |   1.620370% |
|       7 |       38 |   2.932099% |
|       8 |       62 |   4.783951% |
|       9 |       91 |   7.021605% |
|      10 |      122 |   9.413580% |
|      11 |      148 |  11.419753% |
|      12 |      167 |  12.885802% |
|      13 |      172 |  13.271605% |
|      14 |      160 |  12.345679% |
|      15 |      131 |  10.108025% |
|      16 |       94 |   7.253086% |
|      17 |       54 |   4.166667% |
|      18 |       21 |   1.620370% |


